In [2]:
import requests


In [8]:
# Test basic endpoint
response = requests.get("https://allanctan-ai-be-mini-ai.hf.space/")
print("Root:", response.status_code, response.json())



Root: 404 {'detail': 'Not Found'}


In [5]:
# Test health
response = requests.get("https://allanctan-ai-be-mini-ai.hf.space/health")
print("Health:", response.status_code, response.json())

Health: 404 {'detail': 'Not Found'}


In [5]:

url = "https://ymhhvwsjp7v90d-8080.proxy.runpod.net/transcribe/"

# Correct way - actually open and send the file
with open("voice/a_projectile_is.wav", "rb") as audio_file:
    files = {"file": ("a_projectile_is.wav", audio_file, "audio/wav")}
    response = requests.post(url, files=files)
    
print(f"Status: {response.status_code}")
print(f"Response: {response.json()}")

Status: 500


JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [29]:
import json
# Preload the JSON data at module level
try:
    with open("conversation-tree-simplified-with-audio.json", 'r', encoding='utf-8') as file:
        JSON_DATA = json.load(file)
except FileNotFoundError:
    print("❌ Error: File 'all_questions_with_audio.json' not found.")
    JSON_DATA = {}
except json.JSONDecodeError as e:
    print(f"❌ Error: Invalid JSON format - {e}")
    JSON_DATA = {}
except Exception as e:
    print(f"❌ Error: {e}")
    JSON_DATA = {}

In [ ]:
def get_first_question(speaker_name):
    initial_question = JSON_DATA['questions'][0]
    for question in initial_question['children']:
        print(question.get('speaker'))
        if question.get('speaker') == speaker_name:


# get the first question
question = JSON_DATA["questions"][0]
print(question)

# get the first child of the first question
child = question["children"][0]
print(child)

# get the first response of the first child
response = child["responses"][0]
print(response)

{'speaker': 'question', 'message': 'Within the topic of projectile motion, how would you describe a projectile?', 'criteria': ['Answer must define projectile and projectile motion', 'Answer must mention gravity as the sole influence on the object.'], 'children': [{'speaker': 'teacher_2', 'message': 'When we talk about projectile motion, how would you describe a projectile?', 'tag': 'teacher_2_student_2_question_1_0001', 'audio link': 'https://huggingface.co/mon1111/mini-conversations-v2/resolve/main/teacher_2_student_2_question_1_0001.wav?download=true', 'responses': [{'speaker': 'student_2', 'message': "Hmm, okay. So, a projectile is like, anything that gets thrown or shot and then it's just moving through the air.", 'tag': 'teacher_2_student_2_question_1_0002', 'audio link': 'https://huggingface.co/mon1111/mini-conversations-v2/resolve/main/teacher_2_student_2_question_1_0002.wav?download=true', 'responses': [{'speaker': 'teacher_2', 'message': "That's a good start. Think of the main

In [16]:
def start(speaker_name):
    """
    Return message and audio for a speaker from the first level of questions.
    
    Args:
        speaker_name (str): Name of the speaker (e.g., 'student_1', 'teacher_1')
    
    Returns:
        dict: Dictionary containing 'message' and 'audio' for the speaker
    """
    results = []
    
    if 'questions' in JSON_DATA:
        initial_question = JSON_DATA['questions'][0]
        for question in initial_question['children']:
            print(question.get('speaker'))
            if question.get('speaker') == speaker_name:
                result = {
                    'message': question.get('message', ''),
                    'question': initial_question.get('message', ''),
                    'criteria': initial_question.get('criteria', ''),
                    'audio': question.get('audio link', ''),
                }
                return result
    return {
        'message':'Speaker not found',
        'audio': ''
    }


initial_question = start('teacher_2')
print(initial_question)

teacher_2
{'message': 'When we talk about projectile motion, how would you describe a projectile?', 'question': 'Within the topic of projectile motion, how would you describe a projectile?', 'criteria': ['Answer must define projectile and projectile motion', 'Answer must mention gravity as the sole influence on the object.'], 'audio': 'voice/teacher_2_student_2_question_1_0001.wav?download=true'}


In [8]:
current_node = initial_question
question_node = [question for question in JSON_DATA['questions'] if question['message'] == current_node['question']][0]
print(question_node)

{'speaker': 'question', 'message': 'Within the topic of projectile motion, how would you describe a projectile?', 'criteria': ['Answer must define projectile and projectile motion', 'Answer must mention gravity as the sole influence on the object.'], 'children': [{'speaker': 'teacher_2', 'message': 'When we talk about projectile motion, how would you describe a projectile?', 'tag': 'teacher_2_student_2_question_1_0001', 'audio link': 'https://huggingface.co/mon1111/mini-conversations-v2/resolve/main/teacher_2_student_2_question_1_0001.wav?download=true', 'responses': [{'speaker': 'student_2', 'message': "Hmm, okay. So, a projectile is like, anything that gets thrown or shot and then it's just moving through the air.", 'tag': 'teacher_2_student_2_question_1_0002', 'audio link': 'https://huggingface.co/mon1111/mini-conversations-v2/resolve/main/teacher_2_student_2_question_1_0002.wav?download=true', 'responses': [{'speaker': 'teacher_2', 'message': "That's a good start. Think of the main

In [19]:
# loop all the children of question_node, file speaker == 'teacher_1'
responses = []
for child in question_node['children']:
    if child['speaker'] == 'teacher_2':   
        messages = []
        audios = []
        for response in child['responses']:
            # combine the message and audio of 'teacher_1'          
            if response['speaker'] == 'teacher_2':
                messages.append(response['message'])
                audios.append(response['audio'])
            else:
                if messages:
                    responses.append({
                        'message': messages,
                        'audio': audios
                    })
                messages = []
                audios = []
        if messages:
            responses.append({
                'message': messages,
                'audio': audios
            })

print(json.dumps(responses, indent=4))

[]


In [18]:
# loop all responses and combine messages
response_messages = []
for response in responses:
    response_messages.append(' '.join(response['message']))

print(response_messages)




[]


In [65]:
current_node

{'message': 'Define what a projectile is and what projectile motion refers to.',
 'question': 'Define what a projectile is and what projectile motion refers to.',
 'audio': 'https://huggingface.co/datasets/mon1111/audio-files/resolve/main/teacher_1_001.wav?download=true'}

In [23]:

def find_first_speaker_node(data, speaker_name):
    """
    Find the first node that matches the speaker_name by traversing children recursively.
    
    Args:
        data: The JSON data structure (can be a dict or list)
        speaker_name: The speaker name to search for
        
    Returns:
        The first matching node or None if not found
    """
    if isinstance(data, dict):
        # Check if this node matches the speaker
        if data.get('speaker') == speaker_name:
            return data
        
        # Recursively search children
        if 'children' in data:
            for child in data['children']:
                result = find_first_speaker_node(child, speaker_name)
                if result:
                    return result
                    
    elif isinstance(data, list):
        # Search through list items
        for item in data:
            result = find_first_speaker_node(item, speaker_name)
            if result:
                return result
    
    return None

In [42]:
def build_response_prompt(speaker_name, current_node):
    """
    Return prompt to choose the best response from the node.
    """
    # find the question
    question_node = [question for question in JSON_DATA['questions'] if question['message'] == current_node['question']][0]

    # find the responses for the speaker
    responses = []
    for child in question_node['children']:
        if child['speaker'] == speaker_name:   
            response_node = find_first_speaker_node(child['children'], speaker_name)
            responses.append(response_node)

    response_messages = []
    for response in responses:
        response_messages.append(response['message'])

    # build the prompt

    prompt = f""" You are a tutor assessing a student's response. Choose the best response from the list of responses based on the criteria. Respond verbatim from the response.
    Question: {current_node['question']}
    Criteria: {current_node['criteria']}
    Responses: {response_messages}
    """
    return (prompt, responses)

response_prompt = build_response_prompt('teacher_2', initial_question)
print(response_prompt)

(' You are a tutor assessing a student\'s response. Choose the best response from the list of responses based on the criteria. Respond verbatim from the response.\n    Question: Within the topic of projectile motion, how would you describe a projectile?\n    Criteria: [\'Answer must define projectile and projectile motion\', \'Answer must mention gravity as the sole influence on the object.\']\n    Responses: ["That\'s a good start. Think of the main force acting on the object once it has been launched.", \'Correct. Here comes the next question.\', "Remember that a projectile is an object moving through the air that is only affected by one specific force after it\'s launched."]\n    ', [{'speaker': 'teacher_2', 'message': "That's a good start. Think of the main force acting on the object once it has been launched.", 'tag': 'teacher_2_student_2_question_1_0003', 'audio link': 'voice/teacher_2_student_2_question_1_0003.wav', 'children': [{'speaker': 'student_2', 'message': "Oh, right! On

In [2]:
%pip install requests

  Using cached requests-2.32.4-py3-none-any.whl.metadata (4.9 kB)
  Using cached idna-3.10-py3-none-any.whl.metadata (10 kB)
  Using cached urllib3-2.5.0-py3-none-any.whl.metadata (6.5 kB)
  Using cached certifi-2025.7.14-py3-none-any.whl.metadata (2.4 kB)
Using cached requests-2.32.4-py3-none-any.whl (64 kB)
Using cached certifi-2025.7.14-py3-none-any.whl (162 kB)
Using cached idna-3.10-py3-none-any.whl (70 kB)
Using cached urllib3-2.5.0-py3-none-any.whl (129 kB)
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.

[notice] A new release of pip is available: 24.3.1 -> 25.2
[notice] To update, run: python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [1]:
%pip install websockets nest_asyncio

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.

[notice] A new release of pip is available: 24.3.1 -> 25.2
[notice] To update, run: python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [ ]:
# WebSocket Testing in Notebook
# Run this in Jupyter/Colab/any Python notebook

import asyncio
import websockets
import json
import base64
import nest_asyncio

# Enable nested async loops (needed for notebooks)
nest_asyncio.apply()

# Your HF Space WebSocket URL
WS_URL = "wss://allanctan-ai-be-mini-ai.hf.space/ws"  # Replace with your actual URL
AUDIO_FILE = "voice/a_projectile_is.wav"  # Your audio file path

async def test_websocket_connection():
    """Test basic WebSocket connection"""
    try:
        print("Connecting to WebSocket...")
        async with websockets.connect(WS_URL) as websocket:
            print("✅ Connected successfully!")
            
            # Test ping
            print("Sending ping...")
            await websocket.send(json.dumps({"type": "ping"}))
            
            response = await websocket.recv()
            result = json.loads(response)
            print(f"Ping response: {result}")
            
            return True
    except Exception as e:
        print(f"❌ Connection failed: {e}")
        return False

async def test_audio_transcription():
    """Test audio transcription via WebSocket"""
    try:
        print("Connecting for audio test...")
        async with websockets.connect(WS_URL) as websocket:
            print("✅ Connected!")
            
            # Read and encode audio file
            print(f"Reading audio file: {AUDIO_FILE}")
            with open(AUDIO_FILE, "rb") as f:
                audio_data = base64.b64encode(f.read()).decode()
            
            print(f"Audio data length: {len(audio_data)} characters")
            
            # Send audio data
            message = {
                "audio_data": audio_data,
                "mime_type": "audio/wav"
            }
            
            print("Sending audio data...")
            await websocket.send(json.dumps(message))
            
            # Wait for response
            print("Waiting for transcription...")
            response = await websocket.recv()
            result = json.loads(response)
            
            print("📝 Transcription result:")
            print(f"Type: {result.get('type')}")
            print(f"Text: {result.get('text')}")
            
            return result
            
    except FileNotFoundError:
        print(f"❌ Audio file not found: {AUDIO_FILE}")
        print("Please update AUDIO_FILE path or upload an audio file")
        return None
    except Exception as e:
        print(f"❌ Transcription failed: {e}")
        return None

async def test_multiple_messages():
    """Test sending multiple messages in one connection"""
    try:
        print("Testing multiple messages...")
        async with websockets.connect(WS_URL) as websocket:
            
            # Send ping
            await websocket.send(json.dumps({"type": "ping"}))
            response1 = await websocket.recv()
            print(f"Ping 1: {json.loads(response1)}")
            
            # Send another ping
            await websocket.send(json.dumps({"type": "ping"}))
            response2 = await websocket.recv()
            print(f"Ping 2: {json.loads(response2)}")
            
            # Send invalid message
            await websocket.send(json.dumps({"invalid": "message"}))
            response3 = await websocket.recv()
            print(f"Invalid message response: {json.loads(response3)}")
            
            print("✅ Multiple messages test complete")
            
    except Exception as e:
        print(f"❌ Multiple messages test failed: {e}")

# Test functions you can run in notebook cells

# Cell 1: Test basic connection
print("=== Testing WebSocket Connection ===")
result = asyncio.run(test_websocket_connection())

# Cell 2: Test audio transcription  
print("\n=== Testing Audio Transcription ===")
transcription = asyncio.run(test_audio_transcription())

# Cell 3: Test multiple messages
print("\n=== Testing Multiple Messages ===")
asyncio.run(test_multiple_messages())

# Utility function: Upload and test any audio file
def test_uploaded_audio(audio_file_path):
    """Test transcription with a specific audio file"""
    global AUDIO_FILE
    AUDIO_FILE = audio_file_path
    return asyncio.run(test_audio_transcription())

# For Google Colab: Upload file widget
try:
    from google.colab import files
    
    def upload_and_test_audio():
        """Upload audio file in Colab and test transcription"""
        print("Upload your audio file:")
        uploaded = files.upload()
        
        for filename in uploaded.keys():
            print(f"Testing uploaded file: {filename}")
            result = test_uploaded_audio(filename)
            return result
    
    print("\n📁 In Colab, you can run: upload_and_test_audio()")
    
except ImportError:
    print("💡 For local notebook: Update AUDIO_FILE path and run test_audio_transcription()")

# Alternative: Test with base64 string directly
def test_with_base64(base64_audio_string):
    """Test WebSocket with base64 audio string directly"""
    async def test():
        async with websockets.connect(WS_URL) as websocket:
            message = {
                "audio_data": base64_audio_string,
                "mime_type": "audio/wav"
            }
            await websocket.send(json.dumps(message))
            response = await websocket.recv()
            return json.loads(response)
    
    return asyncio.run(test())

print("\n🚀 Run the cells above to test your WebSocket endpoint!")
print(f"📡 Testing URL: {WS_URL}")